In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential, Model
from tensorflow.keras.optimizers import Adam
from collections import deque
import random
from IPython.display import clear_output
import gym
import matplotlib.pyplot as plt

In [15]:
class DQN_Agent:
  def __init__(self, input_shape, n_actions):
    self.rng = 1
    self.rng_min = 0.1
    self.rng_decay = 0.999
    self.discount = 0.95
    self.weights='pacman_ram'
    self.decay_ctr = 0
    self.transfer_weight_ctr = 0

    self.memory = deque(maxlen=1_000_000)

    self.input_shape = input_shape
    self.n_actions = n_actions
    self.model = self.create_model()
    self.target_model = self.create_model()
    self.transfer_weights()

  def create_model(self):
    input = Input(shape=(128,))
    x = Dense(1024, activation='relu')(input)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    V = Dense(1, activation='linear')(x)
    A = Dense(self.n_actions, activation='linear')(x)
    Q = V + tf.subtract(A, tf.reduce_mean(A, axis=1, keepdims=True))
    
    model = Model(inputs=input, outputs=Q)
    model.compile(optimizer=Adam(lr=0.0001), loss='mse')

    return model

  def remember(self, state, action, reward, state_, done):
    self.memory.append([state, action, reward, state_, done])

  def save(self):
    self.model.save_weights(self.weights)

  def load(self):
    self.model.load_weights(self.weights)
    self.transfer_weights()

  def action(self, state):
    if random.random() < self.rng:
      return random.randint(0, self.n_actions - 1)
    else:
      return np.argmax(self.predict(state))

  def predict(self, state):
    return self.model.predict(np.reshape(state, (1, self.input_shape)))

  def train(self):
    if len(self.memory) < 1_000:
      return

    self.decay_ctr += 1
    self.transfer_weight_ctr += 1

    mini_batch = random.sample(self.memory, 32)
    states = np.array([memory[0] for memory in mini_batch])
    states_ = np.array([memory[3] for memory in mini_batch])
    qs = self.model.predict(states)
    qs_ = self.target_model.predict(states_)

    X = states
    y = []

    for i, memory in enumerate(mini_batch):
      action = memory[1]
      reward = memory[2]
      done = memory[4]

      if done:
        q = reward
      else:
        q = reward + self.discount * np.max(qs_[i])

      qs[i][action] = q
      y.append(qs)
    
    self.model.fit(X, np.array(y), verbose=0, shuffle=False)

    if self.decay_ctr > 100:
      self.decay_rng()
      self.decay_ctr = 0

    if self.transfer_weight_ctr > 20_000:
      self.transfer_weights()
      self.transfer_weight_ctr = 0

  def decay_rng(self):
    self.rng = self.rng * self.rng_decay
    if self.rng < self.rng_min:
      self.rng = self.rng_min

  def transfer_weights(self):
    self.target_model.set_weights(self.model.get_weights())

In [16]:
env = gym.make('MsPacman-ram-v0')

In [17]:
agent = DQN_Agent(env.observation_space.shape[0], env.action_space.n)
score_record = []

In [18]:
agent.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 1024)         132096      input_3[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 1024)         1049600     dense_10[0][0]                   
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 512)          524800      dense_11[0][0]                   
____________________________________________________________________________________________

In [ ]:
episodes = 5_000
for episode in range(1, (episodes + 1)):
  state = env.reset()
  done = False
  score = 0
  train_step = 0
  while not done:
    #env.render()
    action = agent.action(state)
    state_, reward, done, info = env.step(action)
    score += reward
    
    train_step += 1
   
    if done:
      reward = -20
    
    agent.remember(state, action, reward, state_, done)
    if train_step > 10:
        agent.train()
        train_step = 0

    if done:
      clear_output(wait=True)
      print(f'Episode: {episode}\nScore: {score}\nAgent RNG: {agent.rng}\nMemory: {len(agent.memory)}')
      score_record.append(score)

    state = state_
env.close()

Episode: 12
Score: 330.0
Agent RNG: 0.994014980014994
Memory: 8426


In [ ]:
plt.plot(score_record)

In [ ]:
def save_score():
  fh = open('score.txt', 'a')
  fh.write(score)
  fh.close()

In [ ]:
len(agent.memory)

In [ ]:
agent.save()